In [1]:
import numpy as np
from collections import Counter

import pandas as pd
import re

Loading data from local computer

In [2]:
Train = pd.read_csv('/Users/computer/OneDrive/Documents/ML github/titanic_train.csv')    #training X and y dataset
Test = pd.read_csv('/Users/computer/OneDrive/Documents/ML github/titanic_test.csv')      #testing dataset (X)
y_test = pd.read_csv('/Users/computer/OneDrive/Documents/ML github/submission.csv')      #testing dataset (y)

In [3]:
full_data = [Train, Test]
y_test.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


Data Preprocessing from Kaggle Link

In [4]:
Train['Has_Cabin'] = Train["Cabin"].apply(lambda x: 0 if type(x) == float else 1)
Test['Has_Cabin'] = Test["Cabin"].apply(lambda x: 0 if type(x) == float else 1)

for dataset in full_data:
    dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1
for dataset in full_data:
    dataset['IsAlone'] = 0   
    dataset.loc[dataset['FamilySize'] == 1, 'IsAlone'] = 1
for dataset in full_data:
    dataset['Embarked'] = dataset['Embarked'].fillna('S')
for dataset in full_data:
    dataset['Fare'] = dataset['Fare'].fillna(Train['Fare'].median())
    
for dataset in full_data:
    age_avg = dataset['Age'].mean()
    age_std = dataset['Age'].std()
    age_null_count = dataset['Age'].isnull().sum()
    age_null_random_list = np.random.randint(age_avg - age_std, age_avg + age_std, size=age_null_count)
    # handling exceptions
    dataset.loc[np.isnan(dataset['Age']), 'Age'] = age_null_random_list
    dataset['Age'] = dataset['Age'].astype(int)
    
def get_title(name):
    title_search = re.search(' ([A-Za-z]+)\.', name)
    #title return
    if title_search:
        return title_search.group(1)
    return ""

for dataset in full_data:
    dataset['Title'] = dataset['Name'].apply(get_title)
    
for dataset in full_data:
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')

    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')  

for dataset in full_data:
    dataset['Sex'] = dataset['Sex'].map({'female' : 0, 'male' : 1}).astype(int)
    title_mapping = { 'Mr' : 1, 'Master' :2, 'Mrs' :3, 'Miss' : 4, 'Rare' : 5}
    dataset['Title'] = dataset['Title'].map(title_mapping)
    dataset['Title'] = dataset['Title'].fillna(0)
    dataset['Embarked'] = dataset['Embarked'].map( {'S' : 0, 'C' : 1, 'Q': 2}).astype(int)

    dataset.loc[ dataset['Fare'] <= 7.91, 'Fare'] 						        = 0
    dataset.loc[(dataset['Fare'] > 7.91) & (dataset['Fare'] <= 14.454), 'Fare'] = 1
    dataset.loc[(dataset['Fare'] > 14.454) & (dataset['Fare'] <= 31), 'Fare']   = 2
    dataset.loc[ dataset['Fare'] > 31, 'Fare'] 							        = 3
    dataset['Fare'] = dataset['Fare'].astype(int)
    
    dataset.loc[ dataset['Age'] <= 16, 'Age'] 					       = 0
    dataset.loc[(dataset['Age'] > 16) & (dataset['Age'] <= 32), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 32) & (dataset['Age'] <= 48), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 48) & (dataset['Age'] <= 64), 'Age'] = 3
    dataset.loc[ dataset['Age'] > 64, 'Age'] 

In [5]:
drop_elements = ['PassengerId', 'Name', 'Ticket', 'Cabin', 'SibSp']          #drop excess columns
Train = Train.drop(drop_elements, axis = 1)
Test = Test.drop(drop_elements, axis = 1)
y_test = y_test.drop('PassengerId', axis = 1)
print(y_test)

     Survived
0           0
1           1
2           0
3           0
4           1
..        ...
413         0
414         1
415         0
416         0
417         1

[418 rows x 1 columns]


In [6]:
# y_train = Train['Survived']
# X_train = Train.drop(y_train)
Train = np.array(Train)                               #making X_train, y_train, X_test, y_test manually
y_train = Train[:,0]
X_train = Train[:, 1:]
X_test = np.array(Test)
print(X_test)
y_test = np.array(y_test)
print(y_test)

[[3 1 2 ... 1 1 1]
 [3 0 2 ... 2 0 3]
 [2 1 3 ... 1 1 1]
 ...
 [3 1 2 ... 1 1 1]
 [3 1 1 ... 1 1 1]
 [3 1 1 ... 3 0 2]]
[[0]
 [1]
 [0]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [1]
 [0]
 [0]
 [1]
 [1]
 [0]
 [1]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [0]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [0]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [1]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [0]
 [1]
 [0]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [1]
 [0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [1]
 [0]
 [1]
 [1]
 [0]
 [1]
 [0]
 [0]
 [1]
 [0]
 [1]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [0]
 [1]
 [0]
 [0]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [0]
 [0]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]


Decision Tree Class

In [38]:
def entropy(y):
    n = np.bincount(y)
    ns = n / len(y)
    return -np.sum([ s * np.log2(s) for s in ns if s>0])

class Node:
    def __init__(
        self, feature=None, threshold=None, left=None, right=None, *, value=None):
        self.f = feature
        self.t = threshold
        self.l = left
        self.r = right
        self.value = value

    def is_leaf_node(self):
        return self.value is not None

class Decision_Tree:
    def __init__(self, min_samples_split=2, max_depth=100, n_feats=None):
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.n_feats = n_feats
        self.root = None
        
    def frequent(self, y):
#         print("y", y)
        if len(y) == 0:
            return
        c = Counter(y)
#         print("c", c)
#         if len(c) == 1:
#             t = c[0]
#         else:
        t = c.most_common()[0][0]    
        return t
        
    def build(self, X, y, depth=0):
        ns, nf = X.shape
        nlab = len(np.unique(y))

        # stopping criteria
        if (
            depth >= self.max_depth
            or nlab == 1
            or ns < self.min_samples_split):
            
            leaf_value = self.frequent(y)
            return Node(value=leaf_value)

        fidx = np.random.choice(nf, self.n_feats, replace=False)

        # selecting best split
        bf, bt = self.bcriteria(X, y, fidx)

        # grow tree
        lidxs, ridxs = self._split(X[:, bf], bt)
        left = self.build(X[lidxs, :], y[lidxs], depth + 1)
        right = self.build(X[ridxs, :], y[ridxs], depth + 1)
        return Node(bf, bt, left, right)
    
    
    def fit(self, X, y):
        self.n_feats = X.shape[1] if not self.n_feats else min(self.n_feats, X.shape[1])
        self.root = self.build(X, y)
    
    
    def bcriteria(self, X, y, fidxs):
        bgain = -1
        si, st = None, None
        for f in fidxs:
            X_c = X[:, f]
            ts = np.unique(X_c)
            for t in ts:
                gain = self.inf_gain(y, X_c, t)

                if gain > bgain:
                    bgain = gain
                    si = f
                    st = t

        return si, st
    
    def inf_gain(self, y, X_c, split_t):
        # parent entropy
        parent_e = entropy(y)

        # split
        lis, ris = self._split(X_c, split_t)

        if len(lis) == 0 or len(ris) == 0:
            return 0

        n = len(y)
        nl, nr = len(lis), len(ris)
        el, er = entropy(y[lis]), entropy(y[ris])
        child_e = (nl / n) * el + (nr / n) * er

        # information gain calculation
        ig = parent_e - child_e
        return ig

    def _split(self, X_column, split_t):
        l_idxs = np.argwhere(X_column <= split_t).flatten()
        r_idxs = np.argwhere(X_column > split_t).flatten()
        return l_idxs, r_idxs
    
    def predict(self, X):
        return np.array([self.traverse_t(x, self.root) for x in X])
 
    def traverse_t(self, x, node):
        if node == None:
            return 0
        if node.is_leaf_node():
            return node.value
#         print("X", x)
#         print(node.value, node)
#         print(node.f, node.t)
        
        z = node.t
#         print("node f", node.f)
#         print("X", x)
        if node.f and node.t != None:
            if x[node.f] <= node.t:
                return self.traverse_t(x, node.l)
        return self.traverse_t(x, node.r)

       


Training and Testing our Decision Tree Model

In [39]:
clf = Decision_Tree()
clf.fit(X_train, y_train)

In [40]:
def accuracy(y_true, y_pred):
    print(len(y_true), len(y_pred))
#     print(y_true, "y_pred", y_pred)
    acc = (sum(a == b for a,b in zip(y_true, y_pred))) / (len(y_true))
    return acc*100
y_pred = clf.predict(X_test)
acc = accuracy(y_test, y_pred)
print("accuracy", acc)

418 418
accuracy [65.78947368]


# Random Forest

In [41]:
def RFfrequent(y):
    z = []
    for i in range(len(y)):
        if type(i) == list:
            i=tuple(i)
        z = np.append(z,i)
    counter = Counter(z)
    most_common = counter.most_common(1)[0][0]
    return most_common

class RandomForest:    
    def __init__(self, nt = 3, min_samples_split = 2, max_depth = 50, n_feats = None):
        self.nt = nt
        print(self.nt)
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.n_feats = n_feats
        self.arrTree = []
     
    def fit(self, X,y):
        self.arrTree = []
        for _ in range (self.nt):
            t = Decision_Tree(min_samples_split = self.min_samples_split,
                             max_depth = self.max_depth, n_feats = self.n_feats)
            nsamp = X.shape[0]
            idx = np.random.choice(nsamp, size = nsamp, replace = True)
            t.fit(X[idx], y[idx])
            self.arrTree.append(t)
    
    def predict(self, X):
        t_preds = np.array([t.predict(X) for t in self.arrTree])
        t_preds = np.swapaxes(t_preds, 0,1)
        print("tree_preds", t_preds)
        y_pred = [RFfrequent(pred) for pred in t_preds]
        return np.array(y_pred)

Training and testing our random forest

In [42]:
clf = RandomForest(nt = 100)
clf.fit(X_train, y_train)

100


In [30]:
y_pred = clf.predict(X_test)
# print("y", y_pred) 
# print("y_t", y_test)
acc = accuracy(y_test, y_pred)
print("accuracy", acc)

tree_preds [[0 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 1 1 ... 1 1 0]]
418 418
accuracy [59.80861244]


Code in Adaboost

# ADABOOST

In [31]:
class Find:
    def __init__(self):
        self.sign = 1
        self.feature_idx = None
        self.threshold = None
        self.alpha = None

    def calculate(self, X):
        n_samples = X.shape[0]
        X_column = X[:, self.feature_idx]
        predictions = np.ones(n_samples)
        if self.sign == 1:
            predictions[X_column < self.threshold] = -1
        else:
            predictions[X_column > self.threshold] = -1

        return predictions


class Adaboost:
    def __init__(self, n_clf=5, lr = 0.01):
        self.n_clf = n_clf
        self.lr = 1e-10
        self.clfs = []

    def fit(self, X, y):
        n_samples, n_features = X.shape

        # weights initialization to 1/N
        w = np.full(n_samples, (1 / n_samples))

        self.clfs = []


        for _ in range(self.n_clf):
            clf = Find()
            min_error = float("inf")

            #finding best threshold and feature
            for feature_i in range(n_features):
                X_column = X[:, feature_i]
                thresholds = np.unique(X_column)

                for threshold in thresholds:
                    s = 1
                    predictions = np.ones(n_samples)
                    predictions[X_column < threshold] = -1

                    # Error 
                    mis = w[y != predictions]
                    error = sum(mis)

                    if error > 0.5:
                        error = 1 - error
                        s = -1

                    # best configuration
                    if error < min_error:
                        clf.sign = s
                        clf.threshold = threshold
                        clf.feature_idx = feature_i
                        min_error = error

            clf.alp = 0.5 * np.log((1.0 - min_error + self.lr) / (min_error + self.lr))

            # update weights
            predictions = clf.calculate(X)

            w *= np.exp(-clf.alp * y * predictions)
            # Normalization
            w /= np.sum(w)

            # Saving classifier
            self.clfs.append(clf)

    def predict(self, X):
        clf_preds = [clf.alp * clf.calculate(X) for clf in self.clfs]
        y_pred = np.sum(clf_preds, axis=0)
        y_pred = np.sign(y_pred)

        return y_pred

Training and Testing Adaboost

In [32]:
model1 = Adaboost(n_clf = 5, lr = 1e-10)
model1.fit(X_train, y_train)

In [33]:
y_pred = model1.predict(X_test)

In [34]:
acc = accuracy(y_test, y_pred)
print("Accuracy:", acc)

418 418
Accuracy: [40.19138756]
